# Analyse of the textual data

Reasearch questions :
* What are the main topic of the ads
* Whats is the favorite topic of each parti
* What are the global sentiment in these ads

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

import warnings; warnings.simplefilter('ignore')
import os, codecs, string, random
import numpy as np
from numpy.random import seed as random_seed
from numpy.random import shuffle as random_shuffle
import matplotlib.pyplot as plt
%matplotlib inline  

seed = 42
random.seed(seed)
np.random.seed(seed)

#NLP libraries
import spacy, nltk, gensim, sklearn
import pyLDAvis.gensim

#Vader
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#Scikit imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [2]:
GENERATED_PATH = '../generated/'
DATA_PATH = '../data/'

vlaams = ['vlaams_belang', 'nva', 'spa', 'cd&v', 'openvld', 'groen', 'pvda']
wallon = ['defi', 'mr', 'ps', 'ecolo', 'ptb', 'cdh']

ads = pd.read_csv(GENERATED_PATH+'clean_ads.csv', index_col='id', parse_dates=['date'])

In [6]:
ads['text'] = ads['text'].astype('str')

In [7]:
# first we need to separate ads in flamish and ads in french
nl_ads = ads[ads.parti.isin(vlaams)]
fr_ads = ads[ads.parti.isin(wallon)]

In [8]:
from spacy.lang.nl.stop_words import STOP_WORDS
chunks = nl_ads.text.str.lower().to_list()

nlp = spacy.load('nl')


processed_docs = list()
for doc in nlp.pipe(chunks, n_threads=5, batch_size=10):

    # Process document using Spacy NLP pipeline.
    ents = doc.ents  # Named entities

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list and keep only words of length 3 or more.
    doc = [token for token in doc if token not in STOP_WORDS and len(token) > 2]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    processed_docs.append(doc)
docs = processed_docs
del processed_docs

/Users/lamborellemaxime/anaconda3/envs/ada/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
/Users/lamborellemaxime/anaconda3/envs/ada/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
/Users/lamborellemaxime/anaconda3/envs/ada/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
/Users/lamborellemaxime/anaconda3/envs/ada/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
/Users/lamborellemaxime/anaconda3/envs/ada/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
/Users/lamborellemax

In [9]:
# Add bigrams too
from gensim.models.phrases import Phrases

# Add bigrams to docs (only ones that appear 15 times or more).
bigram = Phrases(docs, min_count=15)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [10]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]
#MmCorpus.serialize("models/corpus.mm", corpus)

print('Number of unique tokens: %d' % len(dictionary))
print('Number of chunks: %d' % len(corpus))

Number of unique tokens: 1556
Number of chunks: 2776


In [16]:
# models
from gensim.models import LdaMulticore
params = {'passes': 10, 'random_state': seed}
base_models = dict()
model = LdaMulticore(corpus=corpus, num_topics=10, id2word=dictionary, workers=6,
                passes=params['passes'], random_state=params['random_state'])

In [18]:
model.show_topics(num_words=5)

[(0,
  '0.040*"nan" + 0.022*"kies" + 0.019*"leven" + 0.016*"mensen" + 0.016*"politiek"'),
 (1,
  '0.025*"nieuwestrijd" + 0.023*"moeten" + 0.016*"werken" + 0.015*"dag" + 0.012*"zekerheid"'),
 (2,
  '0.020*"like" + 0.019*"mensen" + 0.018*"tijd" + 0.016*"maggie" + 0.014*"block"'),
 (3,
  '0.020*"mensen" + 0.016*"jij" + 0.015*"euro" + 0.014*"hetkananders" + 0.012*"regering"'),
 (4,
  '0.068*"like" + 0.048*"steunt" + 0.040*"like_steunt" + 0.028*"onze" + 0.026*"pagina"'),
 (5,
  '0.024*"singles" + 0.017*"alleenstaanden" + 0.017*"bart" + 0.016*"singles_alleenstaanden" + 0.016*"energie"'),
 (6,
  '0.024*"stem" + 0.023*"onze" + 0.020*"via" + 0.019*"vragen" + 0.018*"jouw"'),
 (7,
  '0.046*"onze" + 0.016*"like" + 0.013*"bent" + 0.011*"terug" + 0.011*"steun"'),
 (8,
  '0.051*"nieuwestrijd" + 0.043*"zekerheid" + 0.029*"opnieuw" + 0.024*"toekomst" + 0.021*"geloven"'),
 (9,
  '0.024*"groen" + 0.022*"onze" + 0.017*"uur" + 0.014*"meld" + 0.012*"meyrem"')]